In [2]:
# !pip install tiktoken

In [1]:
print("hi")

hi


In [3]:

import tiktoken 
tokenizer = tiktoken.get_encoding("gpt2")
with open('poetry.txt', 'r', encoding='latin-1') as f:
    _tokens = tokenizer.encode_ordinary(f.read())

# sample tokens
print("\nSamples from tokenization: ")
print([tokenizer.decode_single_token_bytes(token) for token in _tokens[150:170]])


Samples from tokenization: 
[b' her', b' mother', b' are', b' credited', b' with', b' having', b' researched', b',', b'\n', b'authent', b'icated', b',', b' and', b' compiled', b' much', b' of', b' the', b' material', b' School', b'craft']


In [4]:
num_tokens = len(_tokens)
vocab = list(set(_tokens))
vocab_size = len(vocab)
# ordinal_encodings

otoe = {i : vocab[i] for i in range(vocab_size)}
etoo = {vocab[i] : i for i in range(vocab_size)}
# otoe = {i : _tokens[i] for i in range(num_tokens)}
# etoo = {_tokens[i] : i for i in range(num_tokens)}
ordinalize = lambda t : etoo[t]
deordinalize = lambda t : otoe[t]

tokens = [ordinalize(t) for t in _tokens]
assert(_tokens == [deordinalize(t) for t in tokens])
print(f'number of tokens = {len(tokens)}')
assert(max(tokens) == vocab_size - 1)

number of tokens = 5475758


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

c:\Users\rohan\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
batch_size = 16
block_size = 32
max_iters = 10000
eval_iters = 500
eval_interval = 500
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
n_embd = 32
n_head = 4
n_layer = 4
print("device:" + device)
dropout = 0.5

device:cpu


In [7]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    








class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    






class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    





class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    



class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        # if loss is not None:
        #     if loss < 5.5:  
        #         learning_rate = 1e-3

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [8]:

data = torch.tensor(tokens, dtype=torch.long, device=device)
print(data.shape, data.dtype)

torch.Size([5475758]) torch.int64


In [9]:

train_data = data[:int(num_tokens * 0.9)]
val_data = data[int(num_tokens * 0.9): ]

train_data.get_device()

-1

In [10]:

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch("train")

In [11]:
print(xb.shape)
print(yb.shape)


torch.Size([16, 32])
torch.Size([16, 32])


In [12]:

model = BigramLanguageModel(vocab_size)
m = model.to(device)


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [14]:


# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

2.146535 M parameters


In [15]:
# uncomment this to train a model from scratch
# for x in range(5):
  
#   for iter in range(max_iters):

#       # every once in a while evaluate the loss on train and val sets
#       if iter % eval_interval == 0 or iter == max_iters - 1:
#           losses = estimate_loss()
#           print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#       # sample a batch of dat
#       xb, yb = get_batch('train')

#       # evaluate the loss
#       logits, loss = model(xb, yb)
#       optimizer.zero_grad(set_to_none=True)
#       loss.backward()
#       optimizer.step()

#   model_name = "Model_iter_" + str(50000 + (x+1)*max_iters)
#   torch.save(model.state_dict(), model_name)


In [15]:
vocab_size

32231

In [16]:
model2 = BigramLanguageModel(vocab_size)
m2 = model.to(device)

In [17]:

m2.load_state_dict(torch.load("Model_iter_50000 (1)", map_location=torch.device(device)))

<All keys matched successfully>

In [18]:
# to save the model after training
# torch.save(model.state_dict(), "Model_iter_80000")

In [19]:
xb, yb = get_batch('val')
_idx = model.generate(xb, 100)

print(_idx.shape)

torch.Size([16, 132])


In [20]:

for batch in _idx:
    res = []
    for num in batch:
        num2 = deordinalize(int(num))
        res.append(num2)
    resstr = tokenizer.decode(res)
    print(resstr)


 that stilled Erinna's voice.
Ah, never with a throat that aches with song,
Beneath the white uncaring sky of spring
Afterakes that speech there split Ald "Just as a Haunted goat to good house,
And there, he broke, and down at all night reformation
The fader Rutulchman's mad helmet tender,
To sm next Gerinsaway, and thereby
Is Aldobing, free to her axeth
Resum'd Doctor asked:  how in this, depart, so fallest,
But much halls in loud down theileen.
And the lovely virtues who
cted face
The smiles of love adorn,--
Man's inhumanity to man
Makes countless thousands mourn!
"See yonder poor and th' he arising from this fourth.
Since recompiant, and know angel bears
To heaven the common heart's head he might purge their small memories
DORThose fish were one tears in prime Maker, the golden shoulders, albeit with knots;
Each three'sladen lady's pathways it they rode inside himself,
I seemed unto the labew apace, the eighteen.
Blood-moving round his chestthe ray,
Of roar of thy nakedness him don 

In [21]:

# xb, yb = get_batch('train')
# _idx = model.generate(xb, 100)

# print(_idx.shape)

# for batch in _idx:
#     res = []
#     for num in batch:
#         num2 = deordinalize(int(num))
#         res.append(num2)
#     resstr = tokenizer.decode(res)
#     print(resstr)


torch.Size([16, 132])
, hard as it was,
unavenged must the atheling die!
Too awful it is for an aged man
to bide and bear,
Art thou m worst is their boot, which was craven wit
A ram the wish to changeful, lands, or Sone areving mill,
'Saunt-nen all the music of sunshine by
And keep the waves horse, like lamps,
Their first seemingly thro' "Stand thy Head with Ptried-off murmur.
Thered gates carOn Agugian, they press her former fear and their ways,
Of this gift this gods, was amazed

Native of Thebez, wandering here, was fed
Twice by a voice inviting him to eat.
Of thee those forty days none hath regard
The most amazed the one slew him ethereal deceive.
From its.  All his foes for yearle
This is the sense men should ensue betray the way of line,
Of their accident manner,
Then on the scant battle like his domains he shows
Of earth to be rich nature grow eager eyes uncallsrecian 'What would go lif non other kinde
And who backed axeth faire, and telle kne
Wupuzzered my cause are
 my rural m